<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_N1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **C++**

In [ ]:
%%writefile tp1.cpp
#include <iostream>
#include <unistd.h>
#include <vector>
#include <string>
#include <sys/wait.h>

// Estructura con la que armamos el árbol
typedef struct Node {
  std::string name;
  std::vector<Node> children;
} Node;

int spawnProcesses(Node&);
void processMessage(Node& n){
    std::cout << "Hola mi nombre es " << n.name << ". PID: " << getpid() << " P_PID: "<< getppid() << std::endl;
}

int main()
{
    Node nodo = {"A", {{"B", {{"E"}, {"F"}}}, {"C"}, {"D", {{"G"}}}}};
    spawnProcesses(nodo);

    return EXIT_SUCCESS;
}


// Función recursiva encargada de crear los procesos hijos y esperarlos
int spawnProcesses(Node& n){
    int pid = 0;
    processMessage(n);
    for(auto& c : n.children){
        pid = fork();
        if (pid == -1) {
            perror("fork");
            exit(EXIT_FAILURE);
        } else if(pid == 0){
            spawnProcesses(c);
            sleep(50);
            return EXIT_SUCCESS;
        }
    }

    for(const auto& c : n.children){
        wait(nullptr);
    }
    return 0;
}




Writing tp1.cpp


In [ ]:
!touch resultado_cpp && chmod 700 resultado_cpp

In [ ]:
!g++ tp1.cpp -o tp1.out

In [ ]:
!nohup ./tp1.out 1>resultado_cpp 2>/dev/null &

In [ ]:
!cat resultado_cpp

Hola mi nombre es A. PID: 369 P_PID: 1
Hola mi nombre es B. PID: 370 P_PID: 369
Hola mi nombre es E. PID: 373 P_PID: 370
Hola mi nombre es C. PID: 371 P_PID: 369
Hola mi nombre es F. PID: 374 P_PID: 370
Hola mi nombre es D. PID: 372 P_PID: 369
Hola mi nombre es G. PID: 376 P_PID: 372


In [ ]:
!ps -ef | grep tp1.out

root         369       1  0 15:28 ?        00:00:00 ./tp1.out
root         370     369  0 15:28 ?        00:00:00 ./tp1.out
root         371     369  0 15:28 ?        00:00:00 ./tp1.out
root         372     369  0 15:28 ?        00:00:00 ./tp1.out
root         373     370  0 15:28 ?        00:00:00 ./tp1.out
root         374     370  0 15:28 ?        00:00:00 ./tp1.out
root         376     372  0 15:28 ?        00:00:00 ./tp1.out
root         401     172  0 15:28 ?        00:00:00 /bin/bash -c ps -ef | grep tp1.out
root         403     401  0 15:28 ?        00:00:00 grep tp1.out


In [ ]:
!pstree -pc $(pgrep -f tp1.out -P 1)

tp1.out(369)─┬─tp1.out(370)─┬─tp1.out(373)
             │              └─tp1.out(374)
             ├─tp1.out(371)
             └─tp1.out(372)───tp1.out(376)


# **Python**

In [ ]:
%%writefile tp1.py
import os
import time

def processMessage(nodo):
    mensaje = "Hola mi nombre es {nombre}. PID: {pid}. PPID {ppid}\n"
    print(mensaje.format(nombre=nodo,pid=os.getpid(),ppid=os.getppid()))

def spawnProcesses(nodo):
    pid=0
    nombre,children = nodo
    processMessage(nombre)
    for item in children:
        pid=os.fork()
        if pid < 0:
            print("Error al crear nuevo proceso")
            os._exit(1)
        if not pid:
            spawnProcesses(item)
            time.sleep(200)
            os._exit(0)
    for item in children:
        os.wait()

def main():
    arbol = ("A", [("B", [("E",[]), ("F",[])]), ("C",[]), ("D", [("G",[])])])
    spawnProcesses(arbol)
    os._exit(0)

main()

Overwriting tp1.py


In [ ]:
!touch resultado_py && chmod 700 resultado_py

In [ ]:
!nohup python3 -u tp1.py 1>resultado_py 2>/dev/null &

In [ ]:
!cat resultado_py

Hola mi nombre es A. PID: 14105. PPID 1

Hola mi nombre es C. PID: 14107. PPID 14105

Hola mi nombre es D. PID: 14108. PPID 14105

Hola mi nombre es B. PID: 14106. PPID 14105

Hola mi nombre es F. PID: 14111. PPID 14106
Hola mi nombre es G. PID: 14109. PPID 14108


Hola mi nombre es E. PID: 14110. PPID 14106



In [ ]:
!ps -ef | grep tp1.py

root       14105       1  1 18:21 ?        00:00:00 python3 -u tp1.py
root       14106   14105  0 18:21 ?        00:00:00 python3 -u tp1.py
root       14107   14105  0 18:21 ?        00:00:00 python3 -u tp1.py
root       14108   14105  0 18:21 ?        00:00:00 python3 -u tp1.py
root       14109   14108  0 18:21 ?        00:00:00 python3 -u tp1.py
root       14110   14106  0 18:21 ?        00:00:00 python3 -u tp1.py
root       14111   14106  0 18:21 ?        00:00:00 python3 -u tp1.py
root       14135     147  0 18:21 ?        00:00:00 /bin/bash -c ps -ef | grep tp1.py
root       14137   14135  0 18:21 ?        00:00:00 grep tp1.py


In [ ]:
!pstree -pc $(pgrep -f tp1.py -P 1)

python3(14105)─┬─python3(14106)─┬─python3(14110)
               │                └─python3(14111)
               ├─python3(14107)
               └─python3(14108)───python3(14109)


Comentarios:


*   Al principio python no imprimia el output en el archivo resultado_py porque le faltaba la opcion -u en el comando de ejecucion





# **Java**

In [ ]:
%%writefile Main.java
import java.io.IOException;

public class Main 
{
    public static void main(String[] args) 
    {
      //Construimos Proceso B
      ProcessBuilder processBuilderB = new ProcessBuilder("java", "ProcesoHijo.java", "B");
      processBuilderB.redirectErrorStream(true);
      processBuilderB.inheritIO();

      //Construimos Proceso C
      ProcessBuilder processBuilderC = new ProcessBuilder("java", "ProcesoHijo.java", "C");
      processBuilderC.redirectErrorStream(true);
      processBuilderC.inheritIO();

      //Construimos Proceso D
      ProcessBuilder processBuilderD = new ProcessBuilder("java", "ProcesoHijo.java", "D");
      processBuilderD.redirectErrorStream(true);
      processBuilderD.inheritIO();

      try 
      {
        // Se muestra información del Proceso
        System.out.println(getInfoProcess());

        //Se solicita al SO el inicio del Proceso B, C y D
        Process processB = processBuilderB.start();
        Process processC = processBuilderC.start();
        Process processD = processBuilderD.start();

        //Se espera señal de finalización de lo Procesos B, C y D
        processB.waitFor();
        processC.waitFor();
        processD.waitFor();
        Thread.sleep(10000);
      } 
      //Tratamiento de excepciones
      catch (IOException ex) 
      {
        System.out.println("Error al iniciar Proceso Hijo. Mensaje de error: " + ex.getMessage());
      } 
      catch (InterruptedException ex) 
      {
        System.out.println("Error en realizar waitFor sobre el proceso hijo. Mensaje de error: " + ex.getMessage());
      }
    }

    //Método para obtener información del proceso (PID, PPID)
    public static String getInfoProcess() 
    {
      ProcessHandle processHandle = ProcessHandle.current();
      long pid = processHandle.pid();
      long ppid = processHandle.parent().get().pid();
      return String.format("Proceso A | PID: %s | PPID: %s", pid, ppid);
    }
}

Writing Main.java


In [ ]:
%%writefile ProcesoHijo.java
import java.io.IOException;

public class ProcesoHijo 
{
    //Definimos constantes a utilizar
    private static final String PROCESS_B = "B";
    private static final String PROCESS_D = "D";

    public static void main(String[] args) 
    {
      //Obtenemos el nombre del proceso recibido por parámetro
      String processName = args[0];
      //Mostramos la información del Proceso
      System.out.println("Proceso " + processName + " | " + getInfoProcess());
      switch (processName) 
      {
        //Lógica asociada a la ejecución del Proceso B
        case PROCESS_B:
          execProcessB();
          break;
        //Lógica asociada a la ejecución del Proceso D
        case PROCESS_D:
          execProcessD();
          break;
        default:
          break;
      }
      //Sleep para verificar resultado y tratamiento de excepción 
      try 
      {
        Thread.sleep(10000);
      } 
      catch (InterruptedException ex) 
      {
        System.out.println("Error en sleep. Mensaje: " + ex.getMessage());
      }
    }

    //Método para obtener información del proceso (PID y PPID)
    public static String getInfoProcess() 
    {
      ProcessHandle processHandle = ProcessHandle.current();
      long pid = processHandle.pid();
      long ppid = processHandle.parent().get().pid();
      return String.format("PID: %s | PPID: %s", pid, ppid);
    }

    //Método con las acciones a realizar por el Proceso B
    public static void execProcessB() 
    {
      //Se construye Proceso E
      ProcessBuilder processBuilderE = new ProcessBuilder("java", "ProcesoHijo.java", "E");
      processBuilderE.redirectErrorStream(true);
      processBuilderE.inheritIO();

      //Se construye Proceso F
      ProcessBuilder processBuilderF = new ProcessBuilder("java", "ProcesoHijo.java", "F");
      processBuilderF.redirectErrorStream(true);
      processBuilderF.inheritIO();

      try 
      {
        //Se solicita al SO la ejecución del Proceso E y Proceso F
        Process processE = processBuilderE.start();
        Process processF = processBuilderF.start();

        //Se espera señal de finalización de los procesos E y F
        processE.waitFor();
        processF.waitFor();
      } 
      //Tratamiento de excepciones
      catch (IOException ex) 
      {
        System.out.println("Error al iniciar Proceso Hijo. Mensaje de error: " + ex.getMessage());
      } 
      catch (InterruptedException ex) 
      {
        System.out.println("Error en realizar waitFor sobre el proceso hijo. Mensaje de error: " + ex.getMessage());
      }
    }

    //Método con las acciones a realizar por el Proceso D
    public static void execProcessD() 
    {
      //Se construye Proceso G
      ProcessBuilder processBuilderG = new ProcessBuilder("java", "ProcesoHijo.java", "G");
      processBuilderG.redirectErrorStream(true);
      processBuilderG.inheritIO();

      try 
      {
        //Se solicita al SO la ejecución del Proceso G
        Process processG = processBuilderG.start();

        //Se espera señal de finalización del Proceso G
        processG.waitFor();
      } 
      //Tratamiento de excepciones
      catch (IOException ex) 
      {
        System.out.println("Error al iniciar Proceso Hijo. Mensaje de error: " + ex.getMessage());
      } 
      catch (InterruptedException ex) 
      {
        System.out.println("Error en realizar waitFor sobre el proceso hijo. Mensaje de error: " + ex.getMessage());
      }
    }
}

Writing ProcesoHijo.java


In [ ]:
!nohup java Main.java 1>salidaJava 2>/dev/null &

In [ ]:
!cat salidaJava

Proceso A | PID: 11195 | PPID: 1
Proceso D | PID: 11225 | PPID: 11195
Proceso C | PID: 11222 | PPID: 11195
Proceso B | PID: 11217 | PPID: 11195


In [ ]:
!pstree -pT | grep java

docker-init(1)-+-java(11195)-+-java(11217)-+-java(11316)
               |             |             `-java(11318)
               |             |-java(11222)
               |             `-java(11225)---java(11293)
